## GHRSST NOAA/STAR Metop-B AVHRR FRAC ACSPO v2.80 0.02 L3U Dataset (GDS v2)

**Data summary from UMM metadata:**

>This L3U (Level 3 Uncollated) dataset contains global daily Sea Surface Temperature (SST) on a 0.02 degree grid resolution. It is produced by the National Oceanic and Atmospheric Administration (NOAA) Advanced Clear Sky Processor for Ocean (ACSPO) using L2P (Level 2 Preprocessed) product acquired from the Meteorological Operational satellite B (Metop-B) Advanced Very High Resolution Radiometer 3 (AVHRR/3) (https://podaac.jpl.nasa.gov/dataset/AVHRRF_MB-STAR-L2P-v2.80 ) in Full Resolution Area Coverage (FRAC) mode as input. It is distributed as 10-minute granules in netCDF-4 format, compliant with the Group for High Resolution Sea Surface Temperature (GHRSST) Data Specification version 2 (GDS2). There are 144 granules per 24-hour interval. Fill values are reported in all invalid pixels, including land pixels with >5 km inland. For each valid water pixel (defined as ocean, sea, lake or river), and up to 5 km inland, the following major layers are reported: SSTs and ACSPO clear-sky mask (ACSM; provided in each grid as part of l2p_flags, which also includes day/night, land, ice, twilight, and glint flags). Only input L2P SSTs with QL=5 were gridded, so all valid SSTs are recommended for the users. Per GDS2 specifications, two additional Sensor-Specific Error Statistics layers (SSES bias and standard deviation) are reported in each pixel with valid SST. Ancillary layers include wind speed and ACSPO minus reference Canadian Meteorological Centre (CMC) Level 4 (L4) SST. The ACSPO Metop-B AVHRR FRAC L3U product is monitored and validated against iQuam in situ data (Xu and Ignatov, 2014) in the NOAA SST Quality Monitor (SQUAM) system (Dash et al, 2010). SST imagery and clear-sky mask are evaluated, and checked for consistency with L2P and other satellites/sensors SST products, in the NOAA ACSPO Regional Monitor for SST (ARMS) system. More information about the dataset is found at AVHRRF_MB-STAR-L2P-v2.80 and in (Pryamitsyn et al., 2021).


In [ ]:
center_lat = -20.54953
center_lon = -175.40914
radius_km = 200

In [ ]:
import requests

from urllib.parse import quote

cmr_granule_search_url = "https://cmr.earthdata.nasa.gov/search/granules.json"

params = {
    'short_name': 'AVHRRF_MB-STAR-L3U-v2.80',
    'page_size': 500,
    'temporal': '2022-01-08T00:00:00,2022-01-23T23:59:59',
    'point': f'{center_lon},{center_lat}'
}

response = requests.get(cmr_granule_search_url, params=params)

download_urls = []

if response.status_code == 200:
    data = response.json()
    granules = data['feed']['entry']

    for granule in granules:
        print(f"Granule Title: {granule['title']}")
        print(f"collection_concept_id: {granule['collection_concept_id']}")
        print("Download URLs:")
        for link in granule['links']:
          if 'title' in link.keys() and '.nc' in link['title']:
            print(link['href'])
            download_urls.append(link['href'])
        print()
else:
    print(f"Error {response.status_code}: {response.text}")


---
## Create a directory to hold the data

In [ ]:
from pathlib import Path

data_path = Path.cwd().parent / f"Tonga_sst/{params['short_name']}"
data_path.mkdir(parents=True, exist_ok=True)

In [ ]:
# import shutil

# shutil.rmtree(data_path)

---
## Download the data

In [ ]:
from getpass import getpass
from requests.auth import HTTPBasicAuth
from tqdm.auto import tqdm

username = input('Earthdata username')
password = getpass('Earthdata password')

for url in tqdm(download_urls):
  output_path = data_path / url.split('/')[-1]
  if not output_path.exists():
    !wget --user={username} --password={password} {url} -O {output_path}

del username
del password

---
## Find the paths to the downloaded data

In [ ]:
l2b_paths = list(data_path.glob('*.nc'))
l2b_paths

---
## Subset the data to an AOI defined by a radius around a center coordinate

**Functions used to subset the data to our AOI and sanitize non-utf-8 data attributes**

In [ ]:
import xarray as xr
import numpy as np
from geopy.distance import great_circle

import numpy as np
import xarray as xr
from math import cos, radians

def clip_to_box_around_point_km(dataset, center_lat, center_lon, distance_km, lat_name='lat', lon_name='lon'):
    """
    Clip the dataset to a box around the given WGS84 coordinate (center_lat, center_lon)
    with a box defined by a distance in kilometers from center. distance_km = side length / 2

    Parameters:
    - dataset: xarray Dataset to be clipped.
    - center_lat: Latitude of the center point
    - center_lon: Longitude of the center point
    - distance_km: Distance in kilometers to create a bounding box around the point.
    - lat_name: Name of the latitude coordinate in the dataset.
    - lon_name: Name of the longitude coordinate in the dataset.

    Returns:
    - Clipped xarray Dataset.
    """

    lat_degree_km = 111.32
    delta_lat = distance_km / lat_degree_km
    lon_degree_km = lat_degree_km * cos(radians(center_lat))
    delta_lon = distance_km / lon_degree_km


    min_lat = center_lat - delta_lat
    max_lat = center_lat + delta_lat
    min_lon = center_lon - delta_lon
    max_lon = center_lon + delta_lon

    if min_lon < -180:
        min_lon += 360
    if max_lon > 180:
        max_lon -= 360

    lat = dataset[lat_name]
    lon = dataset[lon_name]

    mask = (
        (lat >= min_lat) & (lat <= max_lat) &
        (lon >= min_lon) & (lon <= max_lon)
    )

    clipped_dataset = dataset.where(mask, drop=True)

    return clipped_dataset
    

def subset_to_haversine_radius(
    dataset: xr.Dataset,
    lat_name: str,
    lon_name: str,
    center_lat: float,
    center_lon: float,
    radius_km: float
    ) -> xr.Dataset:
    """
    Subset an xarray.Dataset to only include points within a given radius using Haversine distance.

    Parameters:
    - dataset (xarray.Dataset): dataset containing latitude and longitude data
    - lat_name (str): name of the latitude variable in the dataset
    - lon_name (str): name of the longitude variable in the dataset
    - center_lat (float): latitude of the center point
    - center_lon (float): longitude of the center point
    - radius_km (float): radius around the center point (in kilometers) to include in the subset.

    Returns (xarray.Dataset): subset dataset with only the points within the specified radius.
    """

    lat = dataset[lat_name].values
    lon = dataset[lon_name].values

    lat_dims = dataset[lat_name].dims
    lon_dims = dataset[lon_name].dims

    # handle flattened data
    if len(lon.shape) == 1 and len(lat.shape) == 1:
        lon, lat = np.meshgrid(lon, lat)

    distance_mask = np.zeros(lon.shape, dtype=bool)

    for i in range(lon.shape[0]):
        for j in range(lon.shape[1]):
            point_lat = lat[i, j]
            point_lon = lon[i, j]
            if not np.isnan(point_lat) and not np.isnan(point_lon):
                distance = great_circle((center_lat, center_lon), (point_lat, point_lon)).kilometers
                if distance <= radius_km:
                    distance_mask[i, j] = True

    if len(lat_dims) > 1:
      distance_mask_da = xr.DataArray(distance_mask, dims=lat_dims, coords={lat_dims[0]: dataset[lat_dims[0]], lat_dims[1]: dataset[lat_dims[1]]})
    else:
      distance_mask_da = xr.DataArray(distance_mask, dims=('lat', 'lon'),  coords={'lat': dataset['lat'], 'lon': dataset['lon']})


    return dataset.where(distance_mask_da) #, drop=True)


def clean_attributes(ds):
    """
    Cleans non-utf-8-encodable data attributes so they can be loaded by xarray

    Parameters:
    - ds (xarray.Dataset): dataset whose attributes may contain non-utf-8-encodable characters

    Returns (xarray.Dataset): dataset whose non-utf-8-encodable attributes have been made utf-8 encodable
    """
    for var in ds.variables:
        for attr, value in ds[var].attrs.items():
            if isinstance(value, str):
                try:
                    ds[var].attrs[attr] = value.encode('utf-8').decode('utf-8')
                except UnicodeEncodeError:
                    ds[var].attrs[attr] = value.encode('utf-8', 'replace').decode('utf-8')
    return ds


**Subset the data**

In [ ]:
from tqdm.auto import tqdm
import xarray as xr

for pth in tqdm(l2b_paths):

  ds = xr.open_dataset(pth, decode_cf=False)

  ds = clean_attributes(ds)


  for var in ds.data_vars:
      if ds[var].dtype in ['int16', 'int32']:
          ds[var] = ds[var].astype('float32')

  ds = xr.decode_cf(ds)


  subset = clip_to_box_around_point_km(ds, center_lat, center_lon, radius_km)

  subset = subset_to_haversine_radius(subset, 'lat', 'lon', center_lat, center_lon, radius_km)

  # use only high quality data
  subset = subset.where(subset['quality_level'] == 5)

  output_pth = pth.parent / f"{pth.name.split('.nc')[0]}_subset.nc"
  subset.to_netcdf(output_pth)

  pth.unlink()

---
# Find the paths to the subset data

In [ ]:
l2b_paths = sorted(list(data_path.glob('*subset.nc')))
l2b_paths

---
## Explore the structure of one of the datasets

In [ ]:
import xarray as xr

ds = xr.open_dataset(l2b_paths[2])
ds

---
## Remove outliers (or don't)

In [ ]:
import numpy as np

min_sst = 999.0
max_sst = 0.0

for pth in l2b_paths:
    ds = xr.open_dataset(pth)
    if not ds['sea_surface_temperature'].isnull().all():
      min_sst = np.nanmin([min_sst, ds['sea_surface_temperature'].quantile(0.0)]) # adjust quantile to remove outliers on the low end
      max_sst = np.nanmax([max_sst, ds['sea_surface_temperature'].quantile(1.0)]) # adjust quantile to remove outliers on the high end

print((min_sst-273.15, max_sst-273.15))

---
## Plot SST distributions

In [ ]:
import matplotlib.pyplot as plt

for pth in l2b_paths:
    ds = xr.open_dataset(pth, mask_and_scale=False)
    ds = ds.where(ds['quality_level'] == 5)
    if not ds['sea_surface_temperature'].isnull().all():
        try:
            plt.hist(ds['sea_surface_temperature'].values.flatten()-273.15, bins=50)
            date_str = f"{ds.attrs['start_time'][:4]}-{ds.attrs['start_time'][4:6]}-{ds.attrs['start_time'][6:8]}"
            plt.title(f'SST Temperature Distribution: {date_str}')
            plt.xlabel('Temperature (°C)')
            plt.ylabel('Frequency')
            plt.show()
        except ValueError:
            pass

---
## Concatenate all time steps into a single Dataset along the 'time' dimension

- drop any empty datasets

In [ ]:
combined_ds = xr.open_mfdataset(l2b_paths, concat_dim='time', combine='nested')
combined_ds = combined_ds.dropna(dim='time', subset=['sea_surface_temperature'], how='all')

In [ ]:
combined_ds

---
## Function to create a time series animation

In [ ]:
# !python -m pip install cartopy

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.img_tiles as cimgt
import cartopy.mpl.ticker as cticker
from matplotlib.animation import FuncAnimation

plt.rcParams['animation.embed_limit'] = 100

%matplotlib notebook

def animate_sst(ds, min_sst, max_sst, interval=200):
    K_CONST = 273.15

    sst = ds['sea_surface_temperature'] - K_CONST  # Convert to Celsius

    min_sst_c = min_sst - K_CONST
    max_sst_c = max_sst - K_CONST

    if np.isnan(sst).all():
        print(f'SST data layer empty. Skipping plot.')
        return

    lat = ds['lat'].values
    lon = ds['lon'].values
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    lon_data = lon_grid.flatten()
    lat_data = lat_grid.flatten()

    fig = plt.figure(figsize=(8, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())

    esri_imagery = cimgt.QuadtreeTiles()
    ax.add_image(esri_imagery, 8)
    ax.coastlines()
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue')

    gl = ax.gridlines(draw_labels=True, crs=ccrs.PlateCarree(), linestyle='--', linewidth=0.5)
    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = cticker.LongitudeFormatter()
    gl.yformatter = cticker.LatitudeFormatter()
    gl.xlabel_style = {'rotation': 30, 'ha': 'right'}
    ax.set_xlabel('Longitude (°)', fontsize=12)
    ax.set_ylabel('Latitude (°)', fontsize=12)

    sst_data = sst.isel(time=0).values.flatten()
    sst_plt = ax.scatter(lon_data, lat_data, c=sst_data, cmap='coolwarm', s=5,
                         transform=ccrs.PlateCarree(), vmin=min_sst_c, vmax=max_sst_c)

    center_lon, center_lat = -175.4, -20.5
    feature_plt = plt.scatter(center_lon, center_lat, color='yellow', marker='*', s=100, label='Tonga Hunga Haapai')

    cbar = plt.colorbar(sst_plt, orientation='vertical', pad=0.05)
    cbar.set_label('Sea Surface Temperature (°C)')

    def update(frame):
        sst_data = sst.isel(time=frame).values.flatten()
        sst_plt.set_array(sst_data)
        date_str = str(ds['time'].values[frame])[:10]
        ax.set_title(f"Sea Surface Temperature (SST), {date_str}\n{params['short_name']}", fontsize=14)
        return sst_plt,

    ani = FuncAnimation(fig, update, frames=len(sst.time), interval=interval, blit=False)

    plt.tight_layout()
    plt.show()

    return ani


---
## Create the SST time series animation

In [ ]:
from IPython.display import display, HTML

anim = animate_sst(combined_ds, min_sst, max_sst, interval=400)
display(HTML(anim.to_jshtml()))

---
## Save the animation as a gif

In [ ]:
from pathlib import Path

animation_dir = data_path.parent / "time_series_animations"
animation_dir.mkdir(parents=True, exist_ok=True)

anim.save(animation_dir/f"{params['short_name']}.gif", writer='pillow', dpi=300)